# Merging

Files needed = (Metro_MedianRentalPrice_1Bedroom.csv, Metro_MedianRentalPrice_Studio.csv, ml-latest-small.zip)

We will often find ourselves with variables spread across different datasets and files. We *merge* datasets together by matching up the two datasets on one or more variables. For example, I might have GDP data by country from the Penn World Tables, and demographic data by country from the World Bank. We would merge these two datasets and match up the observations by country. 

Some of the most powerful analysis comes from combining data from different sources. 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

### The data
We are working with data from [Zillow](https://www.zillow.com/research/data/) on rental listing prices. I would like to see how studio and one bedroom prices differ from each other across cities. Unfortunately, Zillow only lets me download one series at a time. (Hey, it's free data, I'm not complaining...) I have downloaded the studio and one bedroom rental listing pricing and now I would like to create one DataFrame with all the data. 

Time to get to work.

In [ ]:
# The data come as csv files. Load them into separate DatFrames.
studios = pd.read_csv('Metro_MedianRentalPrice_Studio.csv')
onebeds = pd.read_csv('Metro_MedianRentalPrice_1Bedroom.csv')

In [ ]:
# Always check out new data! You never know what could be lurking. 
studios.head(2)
#print(studios.tail())

In [ ]:
onebeds.head(2)
#print(onebeds.tail())

A few things to note. 
1. There are more regions in the one-bedroom data (there are more rows).
2. Time is listed as columns --- this is **wide** data.

What do we want?
1. One DataFrame with one-bedroom and studio prices
2. A multiIndex of regions and time on the rows
3. Columns of studio and one bedroom prices

In [ ]:
# Drop columns we don't need.
studios = studios.drop('SizeRank', axis=1)
onebeds = onebeds.drop('SizeRank', axis=1)

### From wide to long
We could assign indexes and reshape this using `.stack()`. Let's use this as a chance to learn a new command: `.melt()`. `.melt()` is like stack, but does not require the extra steps of creating the multiIndex. It can be a handy function. 

The `melt()` method gets us from a wide DataFrame to a long DataFrame (the [docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.melt.html#pandas.DataFrame.melt)). It moves columns into rows.

### melt

To use `.melt()` we need to choose the *id variables* which are the variables to group the observations by. In this case, we want to group by the region name. 

In [ ]:
studios_melted = pd.melt(studios, id_vars=['RegionName']) # we spec RegionName as the grouping variable
studios_melted.head(20)

### Rename variables

In [ ]:
studios_melted.rename(columns={'RegionName':'region', 'variable':'date', 'value':'studio_price'}, inplace=True)
studios_melted.head()

### Repeat for one bedroom data
The studio data looks great. Get the one bedroom data set up.

In [ ]:
onebeds_melted = pd.melt(onebeds, ['RegionName'])
onebeds_melted.rename(columns={'RegionName':'region', 'variable':'date', 'value':'onebed_price'}, inplace=True)
onebeds_melted.head()

In [ ]:
print(studios_melted.shape)
print(onebeds_melted.shape)

Notice that studios has many fewer observations. 

### Merge the two DataFrames into one DataFrame
We want to match the two DataFrames together according to the region-date pairs. 
In database-ese, we refer to variables we are matching as **keys.** So, in our case, the keys are region and date. 

We also need to tell pandas how to treat keys that are not present in both databases. The different types of 'join' (more database-ese) are
1. **inner**: keep the intersection of the keys
2. **left**: keep all the keys from the left DataFrame
3. **right**: keep all the keys from right DataFrame
4. **outer**: keep all the keys from both DataFrames

We specify the join type with the `how` parameter. The default is inner, but for sanity's sake, be explicit about your join.

In [ ]:
# left and right specify the DataFrames to merge, on specifies the keys (we are using two keys)

rental_prices = pd.merge(left=studios_melted, right=onebeds_melted, on=['region', 'date'], how='inner')
rental_prices

Notice that there are 12,669 rows in the merged DataFrame. That is not equal to the number of rows of either of the original DataFrames. These means there are some region-date pairs in each DataFrame that do not exist in the other. 

Go back and try 'left' and 'right' as `how` types. When you are done, change the merge back to 'inner'. 

### Set the index to region and date

In [ ]:
rental_prices['date'] = pd.to_datetime(rental_prices['date'])   # set the date column to datetime objects

rental_prices.set_index(['region', 'date'], inplace=True) # set up the index and sort it!
rental_prices.sort_index(axis=0, inplace=True)
rental_prices.head()

Abilene does not have much data. Let's check on the US average.

In [ ]:
rental_prices.loc['United States']

### Try an outer merge
Notice that 2010-01 is missing, even though the studio data have an entry for it (look at the result of the `head()` after we melted the studio data). 

Let's try an outer join.

In [ ]:
# The only difference from the previous merge is the switch to 'outer'
rental_prices = pd.merge(left=studios_melted, right=onebeds_melted, on=['region', 'date'], how='outer')

rental_prices['date'] = pd.to_datetime(rental_prices['date'])
rental_prices.set_index(['region', 'date'], inplace=True)
rental_prices.sort_index(axis=0, inplace=True)

rental_prices.loc['United States']

Notice that when we merge as an outer, pandas fills in NaN when there was not an entry in the corresponding DataFrame.

In [ ]:
# Peel off the data for Madison
mad = rental_prices.loc['Madison, WI']

fig, ax = plt.subplots(figsize=(15,8))

ax.plot(mad.index, mad['studio_price'], color='blue')
ax.plot(mad.index, mad['onebed_price'], color='red')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_title('Median rental prices: Madison, WI')
ax.text('06/01/2017', 1050, 'one bedroom')
ax.text('09/10/2017', 925, 'studio')

plt.show()

I wish we had a longer time series. Not much to see here. 

The Florida markets have much longer time series. Let's plot.

In [ ]:

miami = rental_prices.loc['Miami, FL']
tampa = rental_prices.loc['Tampa, FL']

fig, ax = plt.subplots(1, 2, figsize=(15,8))

ax[0].plot(miami.index, miami['studio_price'], color='blue')
ax[0].plot(miami.index, miami['onebed_price'], color='red')

ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)

ax[0].set_title('Median rental prices: Miami, FL')
ax[0].text('03/01/2015', 1400, 'one bedroom')
ax[0].text('09/10/2015', 1960, 'studio')

ax[1].plot(tampa.index, tampa['studio_price'], color='blue')
ax[1].plot(tampa.index, tampa['onebed_price'], color='red')

ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)

ax[1].set_title('Median rental prices: Tampa, FL')
ax[1].text('03/01/2015', 800, 'one bedroom')
ax[1].text('09/10/2015', 1450, 'studio')

plt.show()

Interesting...one bedroom apartments are cheaper than studios. Maybe this reflects studios being located in more desirable neighborhoods? 

Zillow spends a lot of effort trying to predict housing prices. This kind of data is one of the basic inputs into that process. 

## Practice
Head over to [MovieLens](https://grouplens.org/datasets/movielens/) and download the ml-latest-small dataset. It will come as a zipped file. Put it in your user drive (and cwd) and unzip it. You will find 5 files. 


Each user ranks *movies* and can *tag* movies. Users have a `userId` and movies have a `movieId`.
* 'movies.csv' holds the description of movies. An observation is a movie.
* 'ratings.csv' holds user ratings of movies. An observation is a user-movie 
* 'tags.csv' holds the tags a user assigns to a movie. A observation is a user-movie. A user can add more than one tag per movie.


1. Load the 'movies.csv' and the 'ratings.csv' files as DataFrames
2. What size are the two DataFrames?


3. Merge the two files on the movieId. What kind of join should you use? We have several ratings per movie. Call your merged DataFrame 'movie_data'.
4. What size is the resulting DataFrame?

5. Create a histogram of the ratings. Use 10 bins. Use the density=True parameter to plot the density rather than the counts.

The histogram syntax works just like our regular plot command

```python
ax.hist(movie_data['rating'], bins=10, color = 'blue', alpha = 0.25, density=True)
```

6. Load the 'tags.csv' file
7. Merge it into your 'movie_data' DataFrame. What keys should you use? What type of join? Name this DataFrame `movie_data_2`.

Let's get a look at the kinds of tags we have. Try the code below, which introduces us to the `value_counts()` method of DataFrame. It counts the number of occurrences of each unique value. 

In [ ]:
print(movie_data_2['tag'].value_counts())

I don't know what atmospheric means, so let's look at the tag, 'Disney'.
8. Compute the average rating for all movies and for movies tagged 'Disney'


9. Let's see how the ratings of Disney movies compare to all movies. We will plot two histograms on the same axes.
    1. Plot the histogram of movies tagged 'Disney'. Use 10 bins. Make the plot blue.
    2. Plot the histogram of all movies (like you did for question 5). Use 10 bins. make the plot red.
    3. Add a legend
    4. Add the mean ratings for all movies and for superhero movies as text to the histogram